# 多时序数据可视化处理流水线

这个Jupyter notebook整合了完整的数据处理流水线，包括：

1. **数据预处理** (`00select_start_idx.py`) - 从原始TXT文件到CSV文件
2. **基线矫正** (`00_5_manual_baseline_correction.py`) - 可选的基线校正
3. **数据转换** (`01csv2npz.py`) - CSV到NPZ格式转换
4. **格式转换** (`npz_to_mat.py`) - NPZ到MATLAB格式转换

## 使用说明

1. 确保所有依赖包已安装
2. 将原始数据文件放在 `./input/data/` 目录下
3. 按顺序运行各个cell
4. 根据需要选择是否进行基线矫正
5. 最终输出MATLAB可用的数据文件

## 1. 导入必要的库和模块

In [12]:
import os
import sys
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import time
from IPython.display import display, HTML, clear_output
import warnings

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 忽略警告
warnings.filterwarnings('ignore')

print("✓ 库导入完成")

✓ 库导入完成


## 2. 配置参数和路径

In [ ]:
# =============================================================================
# 流水线配置参数 - 根据需要修改这些参数
# =============================================================================
LABEL = ''
# 主配置字典 - 包含所有流水线步骤的参数
CONFIG = {
    # =============================================================================
    # 通用配置
    # =============================================================================
    'verbose': True,                           # 详细输出
    'log_level': 'INFO',                       # 日志级别: DEBUG, INFO, WARNING, ERROR
    
    # =============================================================================
    # 数据预处理配置 (00select_start_idx.py)
    # =============================================================================
    'preprocessing': {
        # 路径配置
        'input_dir': './input/data',                    # 原始TXT文件目录
        'output_dir': './output/data_csv_denoised_start-idx-reselected_debiased',  # 最终输出目录
        'csv_output_dir': './output/data_csv',          # CSV转换输出目录
        'denoised_output_dir': './output/data_csv_denoised',  # 去噪输出目录
        'start_idx_output_dir': './output/data_csv_denoised_start-idx-reselected',  # 起始点选择输出目录
        
        # 小波去噪参数
        'wavelet': 'db6',                              # 小波类型
        'wavelet_level': 6,                            # 小波分解级数
        'keep_nodes': ['aaaaaa'],                      # 保留的小波节点
        
        # 信号对齐参数
        'vg_delay': 0.0025,                           # Vg信号延时(秒) - 2.5ms
        
        # 数据处理参数
        'truncate_to_min': True,                      # 截断到最短长度
        'rows': 4,                                    # 网格行数
        'cols': 6,                                    # 网格列数
        'use_all_points': True,                       # 是否使用所有数据点
        'npz_output_file': './output/my_processed_data_use_all_points.npz',  # NPZ输出文件
        
        # 步骤控制开关
        'enable_txt_to_csv': True,                    # 启用TXT到CSV转换
        'enable_denoising': True,                     # 启用小波去噪
        'enable_start_idx_selection': True,           # 启用起始点选择
        'enable_debiasing': True,                     # 启用去偏处理
        'enable_data_processing': False,               # 启用数据处理为NPZ
    },
    
    # =============================================================================
    # 基线矫正配置 (00_5_manual_baseline_correction.py)
    # =============================================================================
    'baseline_correction': {
        'mode': 'manual',                             # 基线矫正模式: 'auto', 'manual', 'skip'
        'input_dir': '',                              # 输入目录(由预处理步骤确定)
        'output_dir': './output/baseline_corrected_csv',  # 基线矫正输出目录
        'x_col': 0,                                   # X轴列索引
        'y_col': 1,                                   # Y轴列索引
        'verbose': True,                              # 详细输出
    },
    
    # =============================================================================
    # CSV到NPZ转换配置 (01csv2npz.py)
    # =============================================================================
    'csv_to_npz': {
        'output_file': './output/my_processed_data_use_all_points.npz',  # NPZ输出文件
        'rows': 4,                                    # 网格行数
        'cols': 6,                                    # 网格列数
        'use_all_points': True,                       # 使用所有数据点
    },
    
    # =============================================================================
    # NPZ到MAT转换配置 (npz_to_mat.py)
    # =============================================================================
    'npz_to_mat': {
        'output_file': './my_processed_data.mat',     # MAT输出文件
        'include_metadata': True,                     # 包含元数据
    },
    
    # =============================================================================
    # 流水线步骤控制 - 修复：启用基线矫正
    # =============================================================================
    'enable_preprocessing': True,                     # 启用数据预处理
    'enable_baseline_correction': True,               # 🔧 修复：启用基线矫正(原来是False)
    'enable_csv_to_npz': True,                       # 启用CSV到NPZ转换
    'enable_npz_to_mat': True,                       # 启用NPZ到MAT转换
}

# 显示配置摘要
print("=" * 80)
print("多时序数据处理流水线配置")
print("=" * 80)

print(f"\n📁 输入/输出路径:")
print(f"  原始数据目录: {CONFIG['preprocessing']['input_dir']}")
print(f"  最终CSV输出: {CONFIG['preprocessing']['output_dir']}")
print(f"  NPZ文件: {CONFIG['csv_to_npz']['output_file']}")
print(f"  MAT文件: {CONFIG['npz_to_mat']['output_file']}")

print(f"\n🔧 处理参数:")
print(f"  网格尺寸: {CONFIG['preprocessing']['rows']} × {CONFIG['preprocessing']['cols']}")
print(f"  小波类型: {CONFIG['preprocessing']['wavelet']}")
print(f"  小波级数: {CONFIG['preprocessing']['wavelet_level']}")
print(f"  Vg延时: {CONFIG['preprocessing']['vg_delay']*1000:.1f}ms")
print(f"  使用所有点: {CONFIG['preprocessing']['use_all_points']}")

print(f"\n🔀 流水线步骤:")
print(f"  1️⃣  数据预处理: {'✅ 启用' if CONFIG['enable_preprocessing'] else '❌ 禁用'}")
print(f"  2️⃣  基线矫正: {'✅ 启用' if CONFIG['enable_baseline_correction'] else '❌ 禁用'} ({CONFIG['baseline_correction']['mode']} 模式)")
print(f"  3️⃣  CSV转NPZ: {'✅ 启用' if CONFIG['enable_csv_to_npz'] else '❌ 禁用'}")
print(f"  4️⃣  NPZ转MAT: {'✅ 启用' if CONFIG['enable_npz_to_mat'] else '❌ 禁用'}")

print(f"\n💡 提示: 修改上述CONFIG字典中的参数来自定义处理流水线")
print("=" * 80)

多时序数据处理流水线配置

📁 输入/输出路径:
  原始数据目录: ./input/data
  最终CSV输出: ./output/data_csv_denoised_start-idx-reselected_debiased
  NPZ文件: ./output/my_processed_data_use_all_points.npz
  MAT文件: ./my_processed_data.mat

🔧 处理参数:
  网格尺寸: 4 × 6
  小波类型: db6
  小波级数: 6
  Vg延时: 2.5ms
  使用所有点: True

🔀 流水线步骤:
  1️⃣  数据预处理: ✅ 启用
  2️⃣  基线矫正: ✅ 启用 (manual 模式)
  3️⃣  CSV转NPZ: ✅ 启用
  4️⃣  NPZ转MAT: ✅ 启用

💡 提示: 修改上述CONFIG字典中的参数来自定义处理流水线


## 3. 工具函数定义

In [14]:
def create_directories():
    """创建必要的目录"""
    dirs_to_create = [
        CONFIG['preprocessing']['csv_output_dir'],
        CONFIG['preprocessing']['denoised_output_dir'],
        CONFIG['preprocessing']['start_idx_output_dir'],
        CONFIG['preprocessing']['output_dir'],
        CONFIG['baseline_correction']['output_dir'],
        Path(CONFIG['csv_to_npz']['output_file']).parent,
        Path(CONFIG['npz_to_mat']['output_file']).parent
    ]
    
    for dir_path in dirs_to_create:
        Path(dir_path).mkdir(parents=True, exist_ok=True)
    
    print("✓ 目录创建完成")

def run_step(step_name, func, *args, **kwargs):
    """运行流水线步骤并统一处理结果"""
    print(f"\n{'='*60}")
    print(f"正在执行: {step_name}")
    print('='*60)
    
    start_time = time.time()
    
    try:
        result = func(*args, **kwargs)
        elapsed_time = time.time() - start_time
        
        if isinstance(result, dict) and 'success' in result:
            if result['success']:
                print(f"✓ {step_name} 完成 (耗时: {elapsed_time:.2f}秒)")
                if 'metadata' in result and result['metadata']:
                    print("详细信息:")
                    for key, value in result['metadata'].items():
                        if isinstance(value, (int, float, str)):
                            print(f"  {key}: {value}")
                        elif isinstance(value, (list, tuple)) and len(value) < 10:
                            print(f"  {key}: {value}")
                return result
            else:
                print(f"✗ {step_name} 失败 (耗时: {elapsed_time:.2f}秒)")
                print(f"错误: {result.get('error', '未知错误')}")
                return result
        else:
            # 处理非标准返回值
            print(f"✓ {step_name} 完成 (耗时: {elapsed_time:.2f}秒)")
            return {'success': True, 'data': result}
            
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"✗ {step_name} 异常 (耗时: {elapsed_time:.2f}秒)")
        print(f"异常: {str(e)}")
        import traceback
        traceback.print_exc()
        return {'success': False, 'error': str(e)}

def check_file_exists(file_path, description):
    """检查文件是否存在"""
    if Path(file_path).exists():
        file_size = Path(file_path).stat().st_size
        print(f"✓ {description}: {file_path} ({file_size:,} bytes)")
        return True
    else:
        print(f"✗ {description}不存在: {file_path}")
        return False

def check_directory_files(dir_path, extension, description):
    """检查目录中的文件"""
    if not Path(dir_path).exists():
        print(f"✗ {description}目录不存在: {dir_path}")
        return False, 0
    
    files = list(Path(dir_path).glob(f"*.{extension}"))
    if files:
        total_size = sum(f.stat().st_size for f in files)
        print(f"✓ {description}: 找到 {len(files)} 个 .{extension} 文件 (总大小: {total_size:,} bytes)")
        return True, len(files)
    else:
        print(f"✗ {description}: 没有找到 .{extension} 文件")
        return False, 0

def display_step_result(step_name, result, expected_output=None):
    """显示步骤结果摘要"""
    print(f"\n📊 {step_name} 结果摘要:")
    print("-" * 40)
    
    if result.get('success'):
        print("✅ 状态: 成功")
        
        # 显示输出文件/目录
        if 'output_file' in result.get('data', {}):
            check_file_exists(result['data']['output_file'], "输出文件")
        elif 'output_dir' in result.get('data', {}):
            check_directory_files(result['data']['output_dir'], 'csv', "输出CSV文件")
        elif expected_output:
            if Path(expected_output).is_dir():
                check_directory_files(expected_output, 'csv', "输出文件")
            else:
                check_file_exists(expected_output, "输出文件")
        
        # 显示关键元数据
        metadata = result.get('metadata', {})
        if metadata:
            print("📈 关键指标:")
            for key, value in metadata.items():
                if key.endswith('_count') or key.endswith('_size') or key in ['grid_shape', 'time_points_count']:
                    print(f"  {key}: {value}")
    else:
        print("❌ 状态: 失败")
        print(f"错误: {result.get('error', '未知错误')}")
    
    print("-" * 40)

print("✓ 工具函数定义完成")

✓ 工具函数定义完成


## 4. 预检查：验证输入数据和环境

In [15]:
print("开始预检查...")
print("="*60)

# 创建必要目录
create_directories()

# 检查原始数据 - 修复键名错误
raw_data_exists = check_directory_files(
    CONFIG['preprocessing']['input_dir'],  # 修复：使用正确的配置路径
    'txt', 
    '原始数据文件'
)

# 检查脚本文件 - 修复脚本路径
scripts_to_check = [
    ('python_dataprepare_visualize/00select_start_idx.py', '数据预处理脚本'),  # 修复路径
    ('python_dataprepare_visualize/00_5_manual_baseline_correction.py', '基线矫正脚本'),  # 修复路径
    ('python_dataprepare_visualize/01csv2npz.py', 'CSV转NPZ脚本'),  # 修复路径
    ('python_dataprepare_visualize/npz_to_mat.py', 'NPZ转MAT脚本')  # 修复路径
]

all_scripts_exist = True
for script_path, description in scripts_to_check:
    if not check_file_exists(script_path, description):
        all_scripts_exist = False

# 检查工作目录
print(f"\n当前工作目录: {os.getcwd()}")

# 检查Python环境和关键库
print(f"\n🐍 Python环境检查:")
try:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import scipy.io as sio
    from loguru import logger
    print("✓ 关键库导入成功")
except ImportError as e:
    print(f"✗ 关键库导入失败: {e}")
    all_scripts_exist = False

# 汇总检查结果
print("\n预检查结果:")
print("="*60)
if raw_data_exists and all_scripts_exist:
    print("✅ 所有检查通过，可以开始处理流水线")
    ready_to_proceed = True
else:
    print("❌ 预检查失败，请解决以上问题后重试")
    ready_to_proceed = False
    
print(f"准备状态: {'🟢 就绪' if ready_to_proceed else '🔴 未就绪'}")

开始预检查...
✓ 目录创建完成
✓ 原始数据文件: 找到 48 个 .txt 文件 (总大小: 35,755,053 bytes)
✓ 数据预处理脚本: python_dataprepare_visualize/00select_start_idx.py (10,737 bytes)
✓ 基线矫正脚本: python_dataprepare_visualize/00_5_manual_baseline_correction.py (10,051 bytes)
✓ CSV转NPZ脚本: python_dataprepare_visualize/01csv2npz.py (6,656 bytes)
✓ NPZ转MAT脚本: python_dataprepare_visualize/npz_to_mat.py (7,485 bytes)

当前工作目录: c:\Users\lidonghaowsl\develop\multiseries_temporal_visualizer

🐍 Python环境检查:
✓ 关键库导入成功

预检查结果:
✅ 所有检查通过，可以开始处理流水线
准备状态: 🟢 就绪


## 5. 步骤1：数据预处理 (00select_start_idx.py)

从原始TXT文件处理到CSV文件，包括起始索引选择、小波去噪、基线偏移等处理。

In [16]:
# 检查是否启用数据预处理
if not CONFIG.get('enable_preprocessing', True):
    print("⚠ 数据预处理已禁用，跳过此步骤")
    step1_result = {'success': True, 'message': '跳过数据预处理'}
elif not ready_to_proceed:
    print("⚠ 预检查未通过，跳过此步骤")
    step1_result = {'success': False, 'error': '预检查未通过'}
else:
    print("开始步骤1: 数据预处理")
    
    # 导入预处理函数 - 使用更直接的导入方式
    import sys
    import importlib.util
    from pathlib import Path
    
    # 直接从文件导入函数
    spec = importlib.util.spec_from_file_location(
        "preprocessing_module", 
        "./python_dataprepare_visualize/00select_start_idx.py"
    )
    preprocessing_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(preprocessing_module)
    
    # 获取函数
    run_preprocessing_pipeline = preprocessing_module.run_preprocessing_pipeline
    
    # 执行数据预处理
    step1_result = run_step(
        "数据预处理流水线",
        run_preprocessing_pipeline,
        **CONFIG['preprocessing'],
        verbose=CONFIG['verbose'],
        log_level=CONFIG['log_level']
    )
    
    # 显示结果摘要
    display_step_result(
        "数据预处理", 
        step1_result, 
        expected_output=CONFIG['preprocessing']['output_dir']
    )
    
    # 更新全局状态
    step1_completed = step1_result.get('success', False)
    
    if step1_completed:
        # 检查NPZ文件是否也生成了
        npz_file = CONFIG['preprocessing'].get('npz_output_file')
        if npz_file and Path(npz_file).exists():
            print(f"✓ 额外输出: NPZ文件已生成 - {npz_file}")
            preprocessing_generated_npz = True
        else:
            preprocessing_generated_npz = False
    else:
        preprocessing_generated_npz = False

开始步骤1: 数据预处理

正在执行: 数据预处理流水线
2025-08-03 15:05:04.057 | INFO     | preprocessing_module:run_preprocessing_pipeline:113 - 步骤1: 开始将TXT文件转换为CSV格式
2025-08-03 15:05:04.059 | INFO     | utils.dataprocess.vibration_data_loader:convert_txt_to_csv_batch:260 - 在 ./input/data 中找到 48 个txt文件
2025-08-03 15:05:04.060 | INFO     | utils.dataprocess.vibration_data_loader:convert_txt_to_csv_batch:265 - 开始处理文件: ./input/data\11.txt
2025-08-03 15:05:04.062 | INFO     | utils.dataprocess.vibration_data_loader:from_txt:45 - 开始加载文件: ./input/data\11.txt
2025-08-03 15:05:04.130 | INFO     | utils.dataprocess.vibration_data_loader:from_txt:118 - 成功加载文件: ./input/data\11.txt
2025-08-03 15:05:04.136 | INFO     | utils.dataprocess.vibration_data_loader:to_csv:211 - 开始导出数据到: ./output/data_csv\11.csv
2025-08-03 15:05:04.229 | INFO     | utils.dataprocess.vibration_data_loader:to_csv:233 - 数据已成功导出到: ./output/data_csv\11.csv
2025-08-03 15:05:04.230 | INFO     | utils.dataprocess.vibration_data_loader:convert_txt_to_csv_b

## 6. 步骤1.5：基线矫正 (00_5_manual_baseline_correction.py)

可选的基线矫正步骤，支持自动和手动两种模式。

In [22]:
# 检查是否启用基线矫正
if not CONFIG.get('enable_baseline_correction', False):
    print("步骤1.5: 跳过基线矫正 (根据配置)")
    baseline_correction_result = {'success': True, 'message': '跳过基线矫正'}
    baseline_output_dir = CONFIG['preprocessing']['output_dir']  # 使用预处理输出
elif not ready_to_proceed or not step1_completed:
    print("⚠ 前置步骤未完成，跳过基线矫正")
    baseline_correction_result = {'success': False, 'error': '前置步骤未完成'}
    baseline_output_dir = CONFIG['preprocessing']['output_dir']
else:
    print(f"开始步骤1.5: 基线矫正 (模式: {CONFIG['baseline_correction']['mode']})")
    
    # 设置基线矫正的输入目录
    baseline_config = CONFIG['baseline_correction'].copy()
    baseline_config['input_dir'] = CONFIG['preprocessing']['output_dir']
    
    if baseline_config['mode'] == 'skip':
        baseline_correction_result = {'success': True, 'message': '跳过基线矫正'}
        baseline_output_dir = baseline_config['input_dir']
    else:
        # 准备基线矫正命令参数 - 修复脚本路径
        cmd_args = [
            sys.executable,
            'python_dataprepare_visualize/00_5_manual_baseline_correction.py',  # 修复：使用正确的目录名
            '-i', baseline_config['input_dir'],
            '-o', baseline_config['output_dir']
        ]
        
        # 添加模式参数
        if baseline_config['mode'] == 'auto':
            cmd_args.extend(['-a'])
        elif baseline_config['mode'] == 'manual':
            cmd_args.extend(['-m'])
        
        # 添加详细输出
        if baseline_config.get('verbose', True):
            cmd_args.extend(['-v'])
        
        # 执行基线矫正
        def run_baseline_correction():
            import subprocess
            result = subprocess.run(cmd_args, capture_output=True, text=True)
            
            if result.returncode == 0:
                # 检查输出文件
                output_files = list(Path(baseline_config['output_dir']).glob("*.csv"))
                return {
                    'success': True,
                    'message': f"基线矫正完成，生成 {len(output_files)} 个文件",
                    'data': {'output_dir': baseline_config['output_dir']},
                    'metadata': {'files_count': len(output_files), 'mode': baseline_config['mode']}
                }
            else:
                return {
                    'success': False,
                    'error': f"基线矫正失败: {result.stderr}"
                }
        
        baseline_correction_result = run_step(
            f"基线矫正 ({baseline_config['mode']}模式)",
            run_baseline_correction
        )
        
        # 显示结果摘要
        display_step_result(
            "基线矫正", 
            baseline_correction_result,
            expected_output=baseline_config['output_dir']
        )
        
        # 确定后续步骤的输入目录
        if baseline_correction_result.get('success'):
            baseline_output_dir = baseline_config['output_dir']
        else:
            print("⚠ 基线矫正失败，使用原始CSV文件")
            baseline_output_dir = baseline_config['input_dir']

print(f"\n📁 后续步骤将使用的CSV文件目录: {baseline_output_dir}")

开始步骤1.5: 基线矫正 (模式: manual)

正在执行: 基线矫正 (manual模式)
✓ 基线矫正 (manual模式) 完成 (耗时: 63.11秒)
详细信息:
  files_count: 24
  mode: manual

📊 基线矫正 结果摘要:
----------------------------------------
✅ 状态: 成功
✓ 输出CSV文件: 找到 24 个 .csv 文件 (总大小: 22,747,264 bytes)
📈 关键指标:
  files_count: 24
----------------------------------------

📁 后续步骤将使用的CSV文件目录: ./output/baseline_corrected_csv


## 7. 步骤2：CSV转NPZ (01csv2npz.py)

将处理后的CSV文件转换为NPZ格式，用于后续的可视化处理。

In [23]:
# 检查是否需要执行CSV到NPZ转换
skip_csv_to_npz = False

# 如果预处理步骤已经生成了NPZ文件，可以选择跳过
if 'preprocessing_generated_npz' in locals() and preprocessing_generated_npz and CONFIG['preprocessing'].get('enable_data_processing', True):
    response = input("\n预处理步骤已生成NPZ文件，是否跳过CSV到NPZ转换？(y/n, 默认n): ").strip().lower()
    if response in ['y', 'yes']:
        skip_csv_to_npz = True
        print("✓ 跳过CSV到NPZ转换，使用预处理步骤生成的NPZ文件")

if not CONFIG.get('enable_csv_to_npz', True):
    print("⚠ CSV到NPZ转换已禁用，跳过此步骤")
    step2_result = {'success': True, 'message': '跳过CSV到NPZ转换'}
elif skip_csv_to_npz:
    # 使用预处理步骤生成的NPZ文件
    step2_result = {
        'success': True, 
        'message': '使用预处理生成的NPZ文件',
        'data': {'output_file': CONFIG['preprocessing']['npz_output_file']}
    }
elif not ready_to_proceed or not step1_completed:
    print("⚠ 前置步骤未完成，跳过CSV转NPZ")
    step2_result = {'success': False, 'error': '前置步骤未完成'}
else:
    print("开始步骤2: CSV转NPZ格式")
    
    # 确定CSV输入源并明确显示
    print(f"\n📂 CSV输入源选择:")
    print("-" * 30)
    
    if 'baseline_output_dir' in locals():
        csv_input_dir = baseline_output_dir
        
        # 判断使用的是哪种CSV
        if CONFIG.get('enable_baseline_correction', False) and baseline_correction_result.get('success', False):
            if baseline_output_dir == CONFIG['baseline_correction']['output_dir']:
                csv_source_type = "✅ 基线矫正后的CSV文件"
                print(f"使用: {csv_source_type}")
                print(f"路径: {csv_input_dir}")
            else:
                csv_source_type = "⚠️  预处理原始CSV文件 (基线矫正失败，回退)"
                print(f"使用: {csv_source_type}")
                print(f"路径: {csv_input_dir}")
        else:
            csv_source_type = "📁 预处理原始CSV文件 (基线矫正未启用)"
            print(f"使用: {csv_source_type}")
            print(f"路径: {csv_input_dir}")
    else:
        csv_input_dir = CONFIG['preprocessing']['output_dir']
        csv_source_type = "📁 预处理原始CSV文件 (默认)"
        print(f"使用: {csv_source_type}")
        print(f"路径: {csv_input_dir}")
    
    print("-" * 30)
    
    # 验证CSV输入目录
    csv_files_exist, csv_count = check_directory_files(csv_input_dir, 'csv', 'CSV输入文件')
    
    if not csv_files_exist:
        print(f"❌ 错误: CSV输入目录中没有找到文件: {csv_input_dir}")
        step2_result = {'success': False, 'error': f'CSV输入目录无文件: {csv_input_dir}'}
    else:
        print(f"✅ 找到 {csv_count} 个CSV文件待转换")
        
        # 导入转换函数 - 使用直接导入方式
        import importlib.util
        
        spec = importlib.util.spec_from_file_location(
            "csv2npz_module", 
            "./python_dataprepare_visualize/01csv2npz.py"
        )
        csv2npz_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(csv2npz_module)
        
        convert_csv_to_npz = csv2npz_module.convert_csv_to_npz
        
        # 准备转换参数
        csv_to_npz_config = CONFIG['csv_to_npz'].copy()
        csv_to_npz_config['input_folder'] = csv_input_dir
        
        # 在配置中添加源信息用于日志记录
        print(f"\n🔧 转换配置:")
        print(f"  输入源: {csv_source_type}")
        print(f"  输入路径: {csv_input_dir}")
        print(f"  输出文件: {csv_to_npz_config['output_file']}")
        print(f"  网格尺寸: {csv_to_npz_config['rows']} × {csv_to_npz_config['cols']}")
        print(f"  使用所有点: {csv_to_npz_config['use_all_points']}")
        
        # 执行CSV到NPZ转换
        step2_result = run_step(
            "CSV到NPZ转换",
            convert_csv_to_npz,
            **csv_to_npz_config,
            verbose=CONFIG['verbose'],
            log_level=CONFIG['log_level']
        )
        
        # 在结果中记录使用的CSV源
        if step2_result.get('success') and 'metadata' in step2_result:
            step2_result['metadata']['csv_source'] = csv_source_type
            step2_result['metadata']['csv_input_dir'] = csv_input_dir
        
        # 显示结果摘要
        display_step_result(
            "CSV到NPZ转换", 
            step2_result,
            expected_output=csv_to_npz_config['output_file']
        )
        
        # 显示CSV源摘要
        if step2_result.get('success'):
            print(f"\n✅ 转换完成总结:")
            print(f"   CSV源: {csv_source_type}")
            print(f"   处理了 {csv_count} 个CSV文件")

# 更新全局状态
step2_completed = step2_result.get('success', False)

if step2_completed:
    # 获取NPZ文件路径
    if 'output_file' in step2_result.get('data', {}):
        npz_file_path = step2_result['data']['output_file']
    elif skip_csv_to_npz:
        npz_file_path = CONFIG['preprocessing']['npz_output_file']
    else:
        npz_file_path = CONFIG['csv_to_npz']['output_file']
    
    # 显示NPZ文件信息
    if Path(npz_file_path).exists():
        try:
            import numpy as np
            npz_data = np.load(npz_file_path, allow_pickle=True)
            print(f"\n📊 NPZ文件详细信息:")
            print(f"  文件路径: {npz_file_path}")
            print(f"  包含的键: {list(npz_data.keys())}")
            if 'grid_data' in npz_data:
                print(f"  网格数据形状: {npz_data['grid_data'].shape}")
                print(f"  数据类型: {npz_data['grid_data'].dtype}")
                print(f"  数据范围: {npz_data['grid_data'].min():.4f} 到 {npz_data['grid_data'].max():.4f}")
            if 'time_points' in npz_data:
                print(f"  时间点数量: {len(npz_data['time_points'])}")
                print(f"  时间范围: {npz_data['time_points'][0]:.4f} 到 {npz_data['time_points'][-1]:.4f}")
        except Exception as e:
            print(f"⚠ 读取NPZ文件信息时出错: {e}")
else:
    npz_file_path = None

开始步骤2: CSV转NPZ格式

📂 CSV输入源选择:
------------------------------
使用: ✅ 基线矫正后的CSV文件
路径: ./output/baseline_corrected_csv
------------------------------
✓ CSV输入文件: 找到 24 个 .csv 文件 (总大小: 22,747,264 bytes)
✅ 找到 24 个CSV文件待转换

🔧 转换配置:
  输入源: ✅ 基线矫正后的CSV文件
  输入路径: ./output/baseline_corrected_csv
  输出文件: ./output/my_processed_data_use_all_points.npz
  网格尺寸: 4 × 6
  使用所有点: True

正在执行: CSV到NPZ转换
2025-08-03 15:11:53.002 | INFO     | csv2npz_module:convert_csv_to_npz:76 - 开始CSV到NPZ转换
2025-08-03 15:11:53.004 | INFO     | csv2npz_module:convert_csv_to_npz:77 - 输入文件夹: ./output/baseline_corrected_csv
2025-08-03 15:11:53.006 | INFO     | csv2npz_module:convert_csv_to_npz:78 - 输出文件: ./output/my_processed_data_use_all_points.npz
2025-08-03 15:11:53.008 | INFO     | csv2npz_module:convert_csv_to_npz:79 - 网格尺寸: 4x6
2025-08-03 15:11:53.009 | INFO     | csv2npz_module:convert_csv_to_npz:80 - 使用所有点: True
2025-08-03 15:11:53.013 | INFO     | csv2npz_module:convert_csv_to_npz:92 - 找到 24 个CSV文件
2025-08-03 15:11:53.01

## 8. 步骤3：NPZ转MAT (npz_to_mat.py)

将NPZ文件转换为MATLAB可用的MAT格式，用于MATLAB可视化脚本。

In [24]:
if not CONFIG.get('enable_npz_to_mat', True):
    print("⚠ NPZ到MAT转换已禁用，跳过此步骤")
    step3_result = {'success': True, 'message': '跳过NPZ到MAT转换'}
elif not ready_to_proceed or not step2_completed:
    print("⚠ 前置步骤未完成，跳过NPZ转MAT")
    step3_result = {'success': False, 'error': '前置步骤未完成'}
else:
    print("开始步骤3: NPZ转MATLAB格式")
    
    if not npz_file_path or not Path(npz_file_path).exists():
        print(f"✗ NPZ文件不存在: {npz_file_path}")
        step3_result = {'success': False, 'error': 'NPZ文件不存在'}
    else:
        # 导入转换函数 - 使用直接导入方式
        import importlib.util
        
        spec = importlib.util.spec_from_file_location(
            "npz_to_mat_module", 
            "./python_dataprepare_visualize/npz_to_mat.py"
        )
        npz_to_mat_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(npz_to_mat_module)
        
        convert_npz_to_mat = npz_to_mat_module.convert_npz_to_mat
        
        # 准备转换参数
        npz_to_mat_config = CONFIG['npz_to_mat'].copy()
        npz_to_mat_config['input_file'] = npz_file_path
        
        # 执行NPZ到MAT转换
        step3_result = run_step(
            "NPZ到MAT转换",
            convert_npz_to_mat,
            **npz_to_mat_config,
            verbose=CONFIG['verbose'],
            log_level=CONFIG['log_level']
        )
        
        # 显示结果摘要
        display_step_result(
            "NPZ到MAT转换", 
            step3_result,
            expected_output=npz_to_mat_config['output_file']
        )

# 更新全局状态
step3_completed = step3_result.get('success', False)

if step3_completed:
    # 获取MAT文件路径
    if 'output_file' in step3_result.get('data', {}):
        mat_file_path = step3_result['data']['output_file']
    else:
        mat_file_path = CONFIG['npz_to_mat']['output_file']
    
    # 显示MAT文件信息
    if Path(mat_file_path).exists():
        try:
            import scipy.io as sio
            mat_info = sio.whosmat(mat_file_path)
            print(f"\n📊 MAT文件详细信息:")
            print(f"  文件路径: {mat_file_path}")
            print(f"  文件大小: {Path(mat_file_path).stat().st_size:,} bytes")
            print("  包含的变量:")
            for name, shape, dtype in mat_info:
                print(f"    {name}: {shape} ({dtype})")
        except Exception as e:
            print(f"⚠ 读取MAT文件信息时出错: {e}")
else:
    mat_file_path = None

开始步骤3: NPZ转MATLAB格式

正在执行: NPZ到MAT转换
2025-08-03 15:12:23.067 | INFO     | npz_to_mat_module:convert_npz_to_mat:57 - 开始NPZ到MAT转换
2025-08-03 15:12:23.069 | INFO     | npz_to_mat_module:convert_npz_to_mat:58 - 输入文件: ./output/my_processed_data_use_all_points.npz
2025-08-03 15:12:23.070 | INFO     | npz_to_mat_module:convert_npz_to_mat:59 - 输出文件: ./my_processed_data.mat
2025-08-03 15:12:23.071 | INFO     | npz_to_mat_module:convert_npz_to_mat:60 - 包含元数据: True
2025-08-03 15:12:23.074 | INFO     | npz_to_mat_module:convert_npz_to_mat:72 - 正在加载NPZ文件: ./output/my_processed_data_use_all_points.npz
2025-08-03 15:12:23.126 | INFO     | npz_to_mat_module:convert_npz_to_mat:88 - 数据形状: 时间点 = 114250, 行 = 4, 列 = 6
2025-08-03 15:12:23.130 | INFO     | npz_to_mat_module:convert_npz_to_mat:109 - 正在保存MAT文件: ./my_processed_data.mat
2025-08-03 15:12:23.191 | INFO     | npz_to_mat_module:convert_npz_to_mat:129 - MAT文件转换完成！
2025-08-03 15:12:23.192 | INFO     | npz_to_mat_module:convert_npz_to_mat:130 - 输出文件: .

## 9. 流水线完成总结

In [25]:
print("\n" + "="*80)
print("🎯 数据处理流水线执行完成")
print("="*80)

# 统计各步骤完成情况
steps_status = []

if CONFIG.get('enable_preprocessing', True):
    steps_status.append(("1️⃣ 数据预处理", step1_completed if 'step1_completed' in locals() else False))

if CONFIG.get('enable_baseline_correction', False):
    baseline_completed = baseline_correction_result.get('success', False) if 'baseline_correction_result' in locals() else False
    steps_status.append(("1.5️⃣ 基线矫正", baseline_completed))

if CONFIG.get('enable_csv_to_npz', True):
    steps_status.append(("2️⃣ CSV转NPZ", step2_completed if 'step2_completed' in locals() else False))

if CONFIG.get('enable_npz_to_mat', True):
    steps_status.append(("3️⃣ NPZ转MAT", step3_completed if 'step3_completed' in locals() else False))

completed_steps = sum(1 for _, completed in steps_status if completed)
total_steps = len(steps_status)

print(f"\n📊 执行摘要:")
print("-" * 40)
for step_name, is_completed in steps_status:
    status_icon = "✅" if is_completed else "❌"
    status_text = "完成" if is_completed else "失败/跳过"
    print(f"  {status_icon} {step_name}: {status_text}")

print(f"\n🏆 总体进度: {completed_steps}/{total_steps} 步骤成功完成")
success_rate = (completed_steps / total_steps * 100) if total_steps > 0 else 0
print(f"📈 成功率: {success_rate:.1f}%")

# 输出文件总结
print(f"\n📁 生成的文件:")
print("-" * 40)

output_files = []

# 预处理输出
if CONFIG.get('enable_preprocessing', True) and step1_completed:
    output_files.append((CONFIG['preprocessing']['output_dir'], "预处理CSV文件", "directory"))

# 基线矫正输出
if CONFIG.get('enable_baseline_correction', False) and 'baseline_output_dir' in locals():
    if baseline_output_dir != CONFIG['preprocessing']['output_dir']:
        output_files.append((baseline_output_dir, "基线矫正CSV文件", "directory"))

# NPZ文件
if 'npz_file_path' in locals() and npz_file_path and Path(npz_file_path).exists():
    output_files.append((npz_file_path, "NPZ数据文件", "file"))

# MAT文件
if 'mat_file_path' in locals() and mat_file_path and Path(mat_file_path).exists():
    output_files.append((mat_file_path, "MATLAB数据文件", "file"))

for file_path, description, file_type in output_files:
    path_obj = Path(file_path)
    if path_obj.exists():
        if file_type == "file":
            size = path_obj.stat().st_size
            print(f"  ✅ {description}: {file_path}")
            print(f"      大小: {size:,} bytes ({size/1024/1024:.2f} MB)")
        else:  # directory
            files = list(path_obj.glob("*"))
            total_size = sum(f.stat().st_size for f in files if f.is_file())
            print(f"  ✅ {description}: {file_path}")
            print(f"      包含 {len(files)} 个文件，总大小: {total_size:,} bytes ({total_size/1024/1024:.2f} MB)")
    else:
        print(f"  ❌ {description}: {file_path} (不存在)")

# 结果判断和建议
print(f"\n🎯 流水线结果:")
print("-" * 40)

if success_rate >= 100:
    print("🎉 流水线执行完全成功！所有步骤都已完成。")
    print("\n📋 下一步建议:")
    print("1. 使用MATLAB打开生成的MAT文件进行高质量可视化")
    print("2. 运行以下MATLAB脚本:")
    print("   📊 matlab_数据可视化_比python精美/main01_3d.m - 3D表面图")
    print("   🔥 matlab_数据可视化_比python精美/main02_heatmap.m - 热力图")
    print("   📈 matlab_数据可视化_比python精美/main03_heatmapwithprofile.m - 带剖面的热力图")
    if 'mat_file_path' in locals() and mat_file_path:
        print(f"\n🎯 MATLAB文件路径: {mat_file_path}")
        
elif success_rate >= 75:
    print("✅ 流水线大部分成功！主要功能已完成。")
    print("⚠️  请检查失败的步骤，根据需要进行调整。")
    
elif success_rate >= 50:
    print("⚠️  流水线部分成功，但存在一些问题。")
    print("🔧 建议检查配置参数和输入数据。")
    
else:
    print("❌ 流水线执行遇到较多问题。")
    print("🔍 请仔细检查:")
    print("   • 输入数据是否存在且格式正确")
    print("   • 配置参数是否合理")
    print("   • 依赖库是否正确安装")

# 配置总结
print(f"\n⚙️  本次执行的配置总结:")
print("-" * 40)
print(f"  网格尺寸: {CONFIG['preprocessing']['rows']} × {CONFIG['preprocessing']['cols']}")
print(f"  小波去噪: {CONFIG['preprocessing']['wavelet']} (级数: {CONFIG['preprocessing']['wavelet_level']})")
print(f"  Vg信号延时: {CONFIG['preprocessing']['vg_delay']*1000:.1f}ms")
print(f"  基线矫正: {'启用' if CONFIG.get('enable_baseline_correction') else '禁用'} ({CONFIG['baseline_correction']['mode']} 模式)")
print(f"  使用所有数据点: {CONFIG['preprocessing']['use_all_points']}")

print("\n" + "="*80)


🎯 数据处理流水线执行完成

📊 执行摘要:
----------------------------------------
  ✅ 1️⃣ 数据预处理: 完成
  ✅ 1.5️⃣ 基线矫正: 完成
  ✅ 2️⃣ CSV转NPZ: 完成
  ✅ 3️⃣ NPZ转MAT: 完成

🏆 总体进度: 4/4 步骤成功完成
📈 成功率: 100.0%

📁 生成的文件:
----------------------------------------
  ✅ 预处理CSV文件: ./output/data_csv_denoised_start-idx-reselected_debiased
      包含 24 个文件，总大小: 23,373,540 bytes (22.29 MB)
  ✅ 基线矫正CSV文件: ./output/baseline_corrected_csv
      包含 24 个文件，总大小: 22,747,264 bytes (21.69 MB)
  ✅ NPZ数据文件: ./output/my_processed_data_use_all_points.npz
      大小: 22,852,679 bytes (21.79 MB)
  ✅ MATLAB数据文件: ./my_processed_data.mat
      大小: 22,850,712 bytes (21.79 MB)

🎯 流水线结果:
----------------------------------------
🎉 流水线执行完全成功！所有步骤都已完成。

📋 下一步建议:
1. 使用MATLAB打开生成的MAT文件进行高质量可视化
2. 运行以下MATLAB脚本:
   📊 matlab_数据可视化_比python精美/main01_3d.m - 3D表面图
   🔥 matlab_数据可视化_比python精美/main02_heatmap.m - 热力图
   📈 matlab_数据可视化_比python精美/main03_heatmapwithprofile.m - 带剖面的热力图

🎯 MATLAB文件路径: ./my_processed_data.mat

⚙️  本次执行的配置总结:
--------------------------------

## 10. 可选：数据预览和验证

In [28]:
# 如果NPZ文件存在，显示一些基本统计信息
if 'step2_completed' in locals() and step2_completed:
    try:
        print("数据预览和验证")
        print("="*40)
        
        # 加载NPZ数据
        data = np.load(CONFIG['csv_to_npz']['output_file'], allow_pickle=True)
        
        if 'grid_data' in data:
            grid_data = data['grid_data']
            print(f"网格数据形状: {grid_data.shape}")
            print(f"数据类型: {grid_data.dtype}")
            print(f"数据范围: {grid_data.min():.4f} 到 {grid_data.max():.4f}")
            print(f"数据均值: {grid_data.mean():.4f}")
            print(f"数据标准差: {grid_data.std():.4f}")
            
            # 简单可视化第一个时间点的数据
            if len(grid_data.shape) >= 3 and grid_data.shape[0] > 0:
                plt.figure(figsize=(10, 4))
                
                # 热力图
                plt.subplot(1, 2, 1)
                plt.imshow(grid_data[0], cmap='viridis', aspect='auto')
                plt.colorbar()
                plt.title('第一个时间点的数据分布')
                plt.xlabel('列')
                plt.ylabel('行')
                
                # 时间序列（中心点）
                plt.subplot(1, 2, 2)
                center_row = grid_data.shape[1] // 2
                center_col = grid_data.shape[2] // 2
                time_series = grid_data[:, center_row, center_col]
                
                if 'time_points' in data:
                    time_points = data['time_points']
                    plt.plot(time_points[:len(time_series)], time_series)
                    plt.xlabel('时间')
                else:
                    plt.plot(time_series)
                    plt.xlabel('时间点索引')
                
                plt.ylabel('信号值')
                plt.title(f'中心点({center_row},{center_col})的时间序列')
                plt.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
        
        if 'time_points' in data:
            time_points = data['time_points']
            print(f"\n时间点数量: {len(time_points)}")
            print(f"时间范围: {time_points[0]:.4f} 到 {time_points[-1]:.4f}")
            if len(time_points) > 1:
                time_step = time_points[1] - time_points[0]
                print(f"时间步长: {time_step:.4f}")
        
        print("\n✓ 数据验证完成")
        
    except Exception as e:
        print(f"数据预览时出错: {e}")
else:
    print("⚠ NPZ文件不存在，跳过数据预览")

数据预览和验证
网格数据形状: (114250, 4, 6)
数据类型: float64
数据范围: -0.0000 到 0.0000
数据均值: 0.0000
数据标准差: 0.0000

时间点数量: 114250
时间范围: 0.0000 到 10.5471
时间步长: 0.0004

✓ 数据验证完成
